## 容器
#### 我们熟悉的各种层在torch.nn中都是以模型的形式存在，如sigmoid层，或者convolutional层，这些我们熟知的层，在torch.nn中都是以一个模型的形式存在，并且它们都是torch.nn.Module()类的子类。
#### 在实际创建深度学习模型过程中，我们的模型往往需要组合很多层，组合很多层之后的这个模型，在torch.nn结构内跟前面说的内置sigmoid层等，属于同一个类别，都是torch.nn.Module()类的子类。
#### 所以当我们创建一个自己的深度学习模型的时候，同样的也需要去写一个这样的子类，并把需要的层放在这个子类里来组合成一个我们实际需要的模型，这种能把多个层组合到一起来方便使用的东西，就是‘容器’。
#### torch.nn.Module()这个类是torch.nn库中容器的一种，简单来说，容器就是用来把多个神经网络层打包组合成一个方便调用的东西，torch.nn.Module()是torch.nn中最重要的一种容器。
### torch.nn中分别包含如下几种容器：
#### 1) torch.nn.Module类。
最大最复杂的容器
#### 2) torch.nn.Sequential类
以非常简单的步骤生成一个可以训练的模型，输入x之后可以自动对所有内部层进行前向传播，返回预测结果，无需依次对每一层模型调用前向传播。
#### 3) torch.nn.ModuleList类
以列表的形式储存多个深度学习模型。需要传入一个列表来生成实例，列表的每个元素需要是一个模型。可以对这个列表进行象list一样的增删改查。
#### 4) torch.nn.ModuleDict类
以有序字典的形式存储多个深度学习模型。需要传入键值对或者OrderedDict对象来生成实例。可以像dict一样对这个字典进行操作。
#### 5) torch.nn.ParameterList类
以列表的形式保存多个模型参数。原理和可操作范围同上。
#### 6) torch.nn.ParameterDict类
以字典的形式保存多个模型参数。原理和可操作范围同上。

### 1) torch.nn.Module类：

#### 1、使用torch.nn.Module类构造我们自己的深度学习模型，需要编写一个继承于torch.nn.Module的子类，而不是直接调用torch.nn.Module类。
#### 编写子类的过程中，需要对子类的\__init__和forward()方法重写，如下：

In [2]:
import torch
import torch.nn as nn

class SVHN_Model1(nn.Module):
    
    def __init__(self):
        
        #初始化父类的一些属性
        #Python 3 和 Python 2 的另一个区别是: 
        #Python 3 可以使用直接使用 super().xxx 代替 super(Class, self).xxx
        super().__init__()
        
        #先进行2次卷积
        self.cnn = nn.Sequential(
            nn.Conv2d(3,16,kernel_size=(3,3),stride=(2,2)),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16,32,kernel_size=(3,3),stride=(2,2)),
            nn.ReLU(),
            nn.MaxPool2d(2))
        
        #然后链接6个全连接输出层
        self.fc1 = nn.Linear(32*3*7,11)
        self.fc2 = nn.Linear(32*3*7,11)
        self.fc3 = nn.Linear(32*3*7,11)
        self.fc4 = nn.Linear(32*3*7,11)
        self.fc5 = nn.Linear(32*3*7,11)
        self.fc6 = nn.Linear(32*3*7,11)
        
    def forward(self,x):
        
        #对self.cnn使用self.cnn(x)会调用nn.Module的_call_impl()函数，
        #_call_impl()函数内部主要就是self.forward()方法，
        #而nn.Module类本身只有_forward_unimplemented()方法，当调用这个方法时会弹出错误:需要自己定义forward方法后使用
        feat = self.cnn(x)
        
        #view()函数相当于reshape(),-1表示对应位置的维度通过自动计算求得
        #这里是把卷积层的结果传给全链接层，卷积层的结果是带有depth的3维数据，而全链接层的数据都是摊平的2维数据
        #所以需要先view一下，view后的数据维度除了需要保持batch_size之外，其他维度全部合并到一起。
        feat = feat.view(feat.shape[0],-1)
        
        c1 = self.fc1(feat)
        c2 = self.fc2(feat)
        c3 = self.fc3(feat)
        c4 = self.fc4(feat)
        c5 = self.fc5(feat)
        c6 = self.fc6(feat)
        
        return c1,c2,c3,c4,c5,c6
    
svhn_model = SVHN_Model1()

#### 编写子类的时候必须要对__init__()进行重写，并且需要在重写开始位置调用父类(torch.nn.Module)的\__init__()方法。父类的\__init__()方法源码如下：    
    
#### 可以看到torch.nn.Module类的\__init__构造方法中，对很多属性进行了初始化，所以我们需要调用父类的\__init__的方法，以免自己初始化这些属性。

In [ ]:
'''
class Module:
    
    training: bool
    _parameters: Dict[str, Optional[Parameter]]
    _buffers: Dict[str, Optional[Tensor]]
    _non_persistent_buffers_set: Set[str]
    _backward_hooks: Dict[int, Callable]
    _is_full_backward_hook: Optional[bool]
    _forward_hooks: Dict[int, Callable]
    _forward_pre_hooks: Dict[int, Callable]
    _state_dict_hooks: Dict[int, Callable]
    _load_state_dict_pre_hooks: Dict[int, Callable]
    _load_state_dict_post_hooks: Dict[int, Callable]
    _modules: Dict[str, Optional['Module']]
    
    def __init__(self) -> None:
            """
            Initializes internal Module state, shared by both nn.Module and ScriptModule.
            """
            torch._C._log_api_usage_once("python.nn_module")

            """
            Calls super().__setattr__('a', a) instead of the typical self.a = a
            to avoid Module.__setattr__ overhead. Module's __setattr__ has special
            handling for parameters, submodules, and buffers but simply calls into
            super().__setattr__ for all other attributes.
            """
            super().__setattr__('training', True)
            super().__setattr__('_parameters', OrderedDict())
            super().__setattr__('_buffers', OrderedDict())
            super().__setattr__('_non_persistent_buffers_set', set())
            super().__setattr__('_backward_hooks', OrderedDict())
            super().__setattr__('_is_full_backward_hook', None)
            super().__setattr__('_forward_hooks', OrderedDict())
            super().__setattr__('_forward_pre_hooks', OrderedDict())
            super().__setattr__('_state_dict_hooks', OrderedDict())
            super().__setattr__('_load_state_dict_pre_hooks', OrderedDict())
            super().__setattr__('_load_state_dict_post_hooks', OrderedDict())
            super().__setattr__('_modules', OrderedDict())
'''

#### 2、Module类的其他功能

##### add & apply:
##### 添加模型:add_module(name,Module)
##### 对每层应用函数：apply(fn)。将Module里的每一个子模型和Module自身依次作为参数传给自定义的函数

##### modules & parameters:
##### 以生成器的方式返回树状模型中所有的模型，包含子模型，小的组合模型，以及模型自身：modules()
##### 返回模型和子模型的参数：parameters(recurse=True)
##### 获取子模型，获取子模型参数：get_submodule(target),get_parameter(target)

##### children:
##### 返回包含所有子模型的生成器：children()
##### 返回直接子模型的名字和模型：named_children()。

##### transform:
##### 转换：to(device想要转到的设备/dtype想要转换成的数据类型/tensor转换成和给定tensor一样的数据类型和所在设备/memory_format)
##### 将所有的模型参数和缓存数据移动到cpu上：cpu()
##### 参数数据类型转换，bfloat16(),double(),float(),half(),直接修改模型本身

##### reset:
##### 清空模型的参数和缓存：to_empty(device)
##### 将所有参数的导数重置为0：zero_grad(set_to_none = Fasle)

##### other:
##### 切换模型到训练或预测状态：eval(),train(bool)
##### 控制模型是否进行自动导数：requires_grad_(requires_grad=True)
##### 以字典的形式返回模型当前所有的状态：state_dict()

In [4]:
svhn_model

SVHN_Model1(
  (cnn): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=672, out_features=11, bias=True)
  (fc2): Linear(in_features=672, out_features=11, bias=True)
  (fc3): Linear(in_features=672, out_features=11, bias=True)
  (fc4): Linear(in_features=672, out_features=11, bias=True)
  (fc5): Linear(in_features=672, out_features=11, bias=True)
  (fc6): Linear(in_features=672, out_features=11, bias=True)
)

##### 添加模型:add_module(name,Module)

In [5]:
svhn_model.add_module('fc7',nn.Linear(32*3*7,11))
svhn_model

SVHN_Model1(
  (cnn): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=672, out_features=11, bias=True)
  (fc2): Linear(in_features=672, out_features=11, bias=True)
  (fc3): Linear(in_features=672, out_features=11, bias=True)
  (fc4): Linear(in_features=672, out_features=11, bias=True)
  (fc5): Linear(in_features=672, out_features=11, bias=True)
  (fc6): Linear(in_features=672, out_features=11, bias=True)
  (fc7): Linear(in_features=672, out_features=11, bias=True)
)

##### 对每层应用函数：apply(fn)。将Module里的每一个子模型和Module自身依次作为参数传给自定义的函数

In [35]:
#常用来初始化各层的参数
def f(module):
    print(module)

svhn_model.apply(f)

Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2))
ReLU()
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2))
ReLU()
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Sequential(
  (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2))
  (4): ReLU()
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)
Linear(in_features=672, out_features=11, bias=True)
Linear(in_features=672, out_features=11, bias=True)
Linear(in_features=672, out_features=11, bias=True)
Linear(in_features=672, out_features=11, bias=True)
Linear(in_features=672, out_features=11, bias=True)
Linear(in_features=672, out_features=11, bias=True)
Linear(in_features=672, out_features=11, bias=True)
SVHN_Model1(
  (cnn): Sequential(
    (0): Conv2d(3, 1

SVHN_Model1(
  (cnn): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=672, out_features=11, bias=True)
  (fc2): Linear(in_features=672, out_features=11, bias=True)
  (fc3): Linear(in_features=672, out_features=11, bias=True)
  (fc4): Linear(in_features=672, out_features=11, bias=True)
  (fc5): Linear(in_features=672, out_features=11, bias=True)
  (fc6): Linear(in_features=672, out_features=11, bias=True)
  (fc7): Linear(in_features=672, out_features=11, bias=True)
)

##### 返回包含所有子模型的生成器：children()

In [41]:
chi = svhn_model.children()
for i,m in enumerate(chi):
    print(f'第{i}个子模型：\n',m)

第0个子模型：
 Sequential(
  (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2))
  (4): ReLU()
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)
第1个子模型：
 Linear(in_features=672, out_features=11, bias=True)
第2个子模型：
 Linear(in_features=672, out_features=11, bias=True)
第3个子模型：
 Linear(in_features=672, out_features=11, bias=True)
第4个子模型：
 Linear(in_features=672, out_features=11, bias=True)
第5个子模型：
 Linear(in_features=672, out_features=11, bias=True)
第6个子模型：
 Linear(in_features=672, out_features=11, bias=True)
第7个子模型：
 Linear(in_features=672, out_features=11, bias=True)


##### 以生成器的方式返回树状模型中所有的模型，包含子模型，小的组合模型，以及模型自身：modules()

In [78]:
chi2 = svhn_model.modules()

for i,m in enumerate(chi2):
    print(f'第{i}个模型：\n',m)

第0个模型：
 SVHN_Model1(
  (cnn): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=672, out_features=11, bias=True)
  (fc2): Linear(in_features=672, out_features=11, bias=True)
  (fc3): Linear(in_features=672, out_features=11, bias=True)
  (fc4): Linear(in_features=672, out_features=11, bias=True)
  (fc5): Linear(in_features=672, out_features=11, bias=True)
  (fc6): Linear(in_features=672, out_features=11, bias=True)
  (fc7): Linear(in_features=672, out_features=11, bias=True)
)
第1个模型：
 Sequential(
  (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(16, 32, kernel_size=(3

##### 将所有的模型参数和缓存数据移动到cpu上：cpu()

In [ ]:
svhn_model.cpu()

#将所有模型参数和缓存移动到cuda上：cuda(device)
print(torch.cuda.is_available())
print(torch.cuda.current_device)
svhn_model.cuda()

##### 参数数据类型转换，bfloat16(),double(),float(),half(),直接修改模型本身

In [76]:
svhn_model.bfloat16()
svhn_model.double()

SVHN_Model1(
  (cnn): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=672, out_features=11, bias=True)
  (fc2): Linear(in_features=672, out_features=11, bias=True)
  (fc3): Linear(in_features=672, out_features=11, bias=True)
  (fc4): Linear(in_features=672, out_features=11, bias=True)
  (fc5): Linear(in_features=672, out_features=11, bias=True)
  (fc6): Linear(in_features=672, out_features=11, bias=True)
  (fc7): Linear(in_features=672, out_features=11, bias=True)
)

##### 切换模型到训练或预测状态：eval(),train(bool)

In [75]:
svhn_model.eval()
svhn_model.train(True)

SVHN_Model1(
  (cnn): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=672, out_features=11, bias=True)
  (fc2): Linear(in_features=672, out_features=11, bias=True)
  (fc3): Linear(in_features=672, out_features=11, bias=True)
  (fc4): Linear(in_features=672, out_features=11, bias=True)
  (fc5): Linear(in_features=672, out_features=11, bias=True)
  (fc6): Linear(in_features=672, out_features=11, bias=True)
  (fc7): Linear(in_features=672, out_features=11, bias=True)
)

##### 获取子模型，获取子模型参数：get_submodule(target),get_parameter(target)

In [93]:
#target是字符串，书写格式为'父模型名字.子模型名字.子模型名字'，只能是层的名字
#如果target存在，则返回具体的模型或参数，如果不存在或书写错误则报错

#返回svhn_model模型下名字为cnn模型的名字为0的参数
#svhn_model.get_parameter('fc1')

#返回svhn_model模型下名字为cnn模型的名字为0的模型
print(svhn_model.get_submodule('cnn.0'))


Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2))


#####  返回直接子模型的名字和模型：named_children()。

In [95]:
#注意直接两个字
tmp2 = svhn_model.named_children()
for i,m in enumerate(tmp2):
    print(f'第{i}个模型：\n',m)

第0个模型：
 ('cnn', Sequential(
  (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2))
  (4): ReLU()
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
))
第1个模型：
 ('fc1', Linear(in_features=672, out_features=11, bias=True))
第2个模型：
 ('fc2', Linear(in_features=672, out_features=11, bias=True))
第3个模型：
 ('fc3', Linear(in_features=672, out_features=11, bias=True))
第4个模型：
 ('fc4', Linear(in_features=672, out_features=11, bias=True))
第5个模型：
 ('fc5', Linear(in_features=672, out_features=11, bias=True))
第6个模型：
 ('fc6', Linear(in_features=672, out_features=11, bias=True))
第7个模型：
 ('fc7', Linear(in_features=672, out_features=11, bias=True))


##### 返回树状模型的所有模型的名字和模型：named_modules()

In [96]:
tmp3 = svhn_model.named_modules()
for i,m in enumerate(tmp3):
    print(f'第{i}个模型：\n',m)

第0个模型：
 ('', SVHN_Model1(
  (cnn): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=672, out_features=11, bias=True)
  (fc2): Linear(in_features=672, out_features=11, bias=True)
  (fc3): Linear(in_features=672, out_features=11, bias=True)
  (fc4): Linear(in_features=672, out_features=11, bias=True)
  (fc5): Linear(in_features=672, out_features=11, bias=True)
  (fc6): Linear(in_features=672, out_features=11, bias=True)
  (fc7): Linear(in_features=672, out_features=11, bias=True)
))
第1个模型：
 ('cnn', Sequential(
  (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(16, 32, 

##### 返回模型和子模型的参数：parameters(recurse=True)

In [122]:
#参数为True,返回树状模型上所有模型的参数
#参数为Fasle，只返回直接子模型的参数

#返回模型里第二个全连接层的所有参数
tmp4 = svhn_model.fc2.parameters(recurse=True)
for i,m in enumerate(tmp4):
    print(f'第{i}个参数：\n',m.shape)

第0个参数：
 torch.Size([11, 672])
第1个参数：
 torch.Size([11])


##### 控制模型是否进行自动导数：requires_grad_(requires_grad=True)

In [123]:
svhn_model.requires_grad_(False)

SVHN_Model1(
  (cnn): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=672, out_features=11, bias=True)
  (fc2): Linear(in_features=672, out_features=11, bias=True)
  (fc3): Linear(in_features=672, out_features=11, bias=True)
  (fc4): Linear(in_features=672, out_features=11, bias=True)
  (fc5): Linear(in_features=672, out_features=11, bias=True)
  (fc6): Linear(in_features=672, out_features=11, bias=True)
  (fc7): Linear(in_features=672, out_features=11, bias=True)
)

##### 以字典的形式返回模型当前所有的状态：state_dict()

In [137]:
#也可以用来查看某一个具体参数

print(list(svhn_model.state_dict().keys()),'\n')
print(list(svhn_model.state_dict().values())[1].shape)
print(list(svhn_model.state_dict().values())[1])

['cnn.0.weight', 'cnn.0.bias', 'cnn.3.weight', 'cnn.3.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias', 'fc4.weight', 'fc4.bias', 'fc5.weight', 'fc5.bias', 'fc6.weight', 'fc6.bias', 'fc7.weight', 'fc7.bias'] 

torch.Size([16])
tensor([ 0.0952, -0.0598, -0.1001, -0.1191,  0.1641, -0.1553,  0.0522, -0.0344,
        -0.1016,  0.1299,  0.1484,  0.1045,  0.1875,  0.0398,  0.0874, -0.1465],
       dtype=torch.float64)


##### 转换：to(device想要转到的设备/dtype想要转换成的数据类型/tensor转换成和给定tensor一样的数据类型和所在设备/memory_format)

In [141]:
#转换直接在原数据上修改

svhn_model.fc2.weight.to(torch.int)

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32)

##### 清空模型的参数和缓存：to_empty(device)

##### 将所有参数的导数重置为0：zero_grad(set_to_none = Fasle)

In [143]:
svhn_model.zero_grad(True)